In [1]:
import sys
import time
import subprocess
import signal
import os
import math
import rospy
import rosbag
import roslib
roslib.load_manifest("robot_comm")
from robot_comm.srv import *
roslib.load_manifest("Mocap")
from Mocap.msg import *
from Mocap.srv import *

In [2]:
get_cart = rospy.ServiceProxy('/robot_GetCartesian', robot_GetCartesian)
set_work_object = rospy.ServiceProxy('/robot_SetWorkObject', robot_SetWorkObject)
set_tool = rospy.ServiceProxy('/robot_SetTool', robot_SetTool)
set_cart = rospy.ServiceProxy('/robot_SetCartesian', robot_SetCartesian)
#Set Speed
set_speed = rospy.ServiceProxy('/robot_SetSpeed', robot_SetSpeed)
set_speed(50, 5)
get_cart()

x: 611.78
y: 310.74
z: 29.42
q0: 0.0004
qx: -0.7058
qy: -0.7084
qz: 0.0004
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [2]:
set_mocap_tf = rospy.ServiceProxy('/mocap_SetMocapTransformation', mocap_SetMocapTransformation)
set_mocap_tf(686.48, -281.16, 62.547, 0.5053, 0.50804, 0.50412, 0.48212)
get_mocap_frame = rospy.ServiceProxy('/mocap_GetMocapFrame', mocap_GetMocapFrame)

In [3]:
get_mocap_frame()

mf: 
  header: 
    seq: 0
    stamp: 
      secs: 1431890523
      nsecs: 319991421
    frame_id: ''
  number: 0
  body_poses: 
    - 
      position: 
        x: 371.767710423
        y: -85.558318326
        z: 46.7418066145
      orientation: 
        x: 0.705380808341
        y: 0.0869412014905
        z: 0.102945110771
        w: 0.695910096415
  uid_markers: 
    markers: 
      - 
        x: 377.622264838
        y: -121.242571165
        z: 47.185764473
      - 
        x: 354.921235856
        y: -75.7489710402
        z: 46.3688407049
      - 
        x: 382.744819311
        y: -59.7400077638
        z: 46.6711307966
  id_marker_sets: 
    - 
      markers: 
        - 
          x: 377.627505338
          y: -121.297954459
          z: 47.1863910989
        - 
          x: 354.847329948
          y: -75.7579658212
          z: 46.3677097536
        - 
          x: 382.828295323
          y: -59.6190643786
          z: 46.67132516
ret: 1
msg: OK.

In [5]:
set_work_object(0,0,0,1,0,0,0)
set_tool(-100,0,110, 0, 0, 1, 0)
get_cart()

x: 600.0
y: 0.01
z: 25.0
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0001
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [22]:
set_cart(600, 0, 20, 1, 0, 0, 0)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [15]:
set_speed(3,5)
#p = subprocess.Popen("rosbag record -O test.bag /Mocap /netft_data");

In [23]:
f1 = open("pos5.txt", "w");
f2 = open("force5.txt", "w");
f3 = open("robot5.txt", "w");
bag = rosbag.Bag('robot_push_5.bag')
last_topic = '/netft_data'
ct = 0
last_pos_x = 0
last_pos_y = 0
pose = geometry_msgs.msg.Pose()
force = geometry_msgs.msg.Wrench.force
for topic, msg, t in bag.read_messages(topics=['/Mocap', '/netft_data', '/robot_CartesianLog']):
    s = str(t.to_sec()) + '\n'
    if topic == '/Mocap':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation    
        s = s + str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);  
    if topic == '/netft_data':
        force = msg.wrench.force
        s = s + str(force.x) + ' ' + str(force.y) + ' ' + str(force.z)
        s = s + '\n'
        f2.write(s)
    if topic == '/robot_CartesianLog':
        robot_cart = msg
        s = s + str(robot_cart.x) + ' ' + str(robot_cart.y) + ' ' + str(robot_cart.z) + ' '
        s = s + str(robot_cart.q0) + ' ' + str(robot_cart.qx) + ' ' + str(robot_cart.qy) + ' ' + str(robot_cart.qz)
        s = s + '\n'
        f3.write(s)
#     if topic == '/Mocap' and last_topic == '/netft_data':
#         pos = msg.body_poses[0].position
#         ori = msg.body_poses[0].orientation    
#         s = str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
#         s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
#         s = s + '\n'
#         f1.write(s);  
#     if topic == '/netft_data' and last_topic == '/Mocap':
#         force = msg.wrench.force
#         s = str(force.x) + ' ' + str(force.y) + ' ' + str(force.z)
#         s = s + '\n'
#         f2.write(s)
#     last_topic = topic
bag.close()
f1.close()
f2.close()
f3.close()

x: 0.0
y: 0.0
z: 0.0

In [11]:
# Just dump the raw mocap body data.
f1 = open("mocap_pos_5.txt", "w")
bag = rosbag.Bag('test_human_push_5.bag')
for topic, msg, t in bag.read_messages(topics=['/Mocap']):
    if topic == '/Mocap':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation
        s = str(t.to_sec()) + '\n'
        s = s + str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);
f1.close()

In [199]:
t.to_sec()

1429908519.3448133

In [122]:
set_tool(-100, 0, 110, 0, 0, 1, 0);


In [121]:
set_cart(550, -100, 375, 1, 0, 0, 0)

x: 549.99
y: -99.97
z: 375.0
q0: 0.0
qx: -0.7071
qy: 0.7071
qz: 0.0001
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [123]:
get_joints = rospy.ServiceProxy('/robot_GetJoints', robot_GetJoints)

In [124]:
get_joints()

j1: 0.0
j2: 20.55
j3: 0.92
j4: 0.01
j5: 68.54
j6: 90.0
ret: 1
msg: ROBOT_CONTROLLER: OK.